In [3]:
from pathlib import Path
import sys

import pandas as pd
from sklearn.tree import DecisionTreeClassifier

rank = 0 
sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, grid_serch_cv

# 特徴量の比較について
 - ホールドアウト法による特徴の比較
 - グリッドサーチを加えた特徴量の比較

### ホールドアウト法による特徴量の比較
３種類の特徴量を同じモデルで比較する

In [4]:
########################## 3種類のパイプラインを用意 ##############################
def pipe_1(df, valid, model, valid_args, params, view=False):
    pipe = PipeLine()
    pipe(df)
    model = pipe.training(valid, model, valid_args, params, view)
    return model

def pipe_2(df, valid, model, valid_args, params, view=False):
    pipe = PipeLine()
    pipe(df)
    pipe.standard_scaler()
    model = pipe.training(valid, model, valid_args ,params, view)
    return model

def pipe_3(df, valid, model, valid_args, params, view):
    pipe = PipeLine()
    pipe(df)
    pipe.standard_scaler()
    pipe.one_hot(['ChestPainType'])
    model = pipe.training(valid, model, valid_args ,params, view)
    return model

### 出力結果

In [5]:
file_path = '../data/train.csv'
df = pd.read_csv(file_path)

############################# 用意した関数に渡す引数 ##############################
pipe_kwargs = {'df': df,
        'valid': 'fold_out_split',  # ホールドアウト法
        'model': DecisionTreeClassifier,  # sklearnのモデル
        'valid_args': {},  # テストサイズの割合を指定
        'params': {'random_state': 42},  # 選択したモデルパラメータを入力
        'view': False}

######################## 用意したパイプラインをfor文で訓練 #########################
pipe_lines = [pipe_1, pipe_2, pipe_3]  # 作ったdemo関数をリストでまとめる （）いらない
models = []  # 訓練したモデルの格納先
for pipe in pipe_lines:
    print('#'*20,f'{pipe.__name__}の結果', '#'*20)  # 表示結果のラベル
    model = pipe(**pipe_kwargs)
    models.append(model)
    print()

#################### pipe_1の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,1.000000,1.000000,1.000000,1.000000
test,0.761658,0.818966,0.791667,0.805085



#################### pipe_2の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,1.000000,1.000000,1.0,1.000000
test,0.766839,0.820513,0.8,0.810127



#################### pipe_3の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,1.000000,1.000000,1.000000,1.000000
test,0.715026,0.764228,0.783333,0.773663


pipe_1とpipe_2は同じような特徴量。決定木だから標準化の単位を受けなから当然<br>
pipe_3の特徴量についてはむしろ下がっている<br>
すべてのモデルで過学習してますね

### グリッドサーチを加えた特徴量の比較

In [9]:
###################### グリッドサーチ用のパイプラインを用意 ########################
def pipe_4(df, model, param_grid):
    pipe = PipeLine()
    pipe(df)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

def pipe_5(df, model, param_grid):
    pipe = PipeLine()
    pipe(df)
    pipe.standard_scaler()
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

def pipe_6(df, model, param_grid):
    pipe = PipeLine()
    pipe(df)
    pipe.standard_scaler()
    pipe.one_hot(['ChestPainType'])
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

### 出力結果

In [10]:
#################### グリッドサーチの探索候補を用意 #################
param_grid = {'max_depth':[1, 2, 3, 4, 5],
              'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'min_samples_split':[2, 3, 4, 5],
              'random_state': [42]}

############################# 用意した関数に渡す引数 ##############################
pipe_kwargs = {'df': df,
               'model': DecisionTreeClassifier,  # sklearnのモデル
               'param_grid': param_grid}

######################## 用意したパイプラインをfor文で訓練 #########################
pipe_lines = [pipe_4, pipe_5, pipe_6]  # 作ったdemo関数をリストでまとめる （）いらない
models = []  # 訓練したモデルの格納先
for pipe in pipe_lines:
    print('#'*20,f'{pipe.__name__}の結果', '#'*20)  # 表示結果のラベル
    model = pipe(**pipe_kwargs)
    models.append(model)
    print()

#################### pipe_4の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.826281,0.828571,0.849372,0.838843
test,0.797927,0.852174,0.816667,0.834043



#################### pipe_5の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.826281,0.828571,0.849372,0.838843
test,0.797927,0.852174,0.816667,0.834043



#################### pipe_6の結果 ####################
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.859688,0.889381,0.841004,0.864516
test,0.740933,0.812500,0.758333,0.784483


全体的に精度が向上している。過学習も抑えられているようで。<br>
にしてもChestPainType単体でワンホットは精度を下げるよう<br>
特徴量については相性があるから他の特徴量との組み合わせも考慮してみるとアリかも